<a href="https://colab.research.google.com/github/himethma/Fine-Tuned-Phi-2-on-Developer-Docs/blob/main/finetune_phi2_lamini_docs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Dependencies

In [ ]:
!pip install -q accelerate peft bitsandbytes transformers trl datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Import dependencies

In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline
)
from peft import LoraConfig, get_peft_model, TaskType
from trl import SFTTrainer
from datasets import Dataset
import json

In [ ]:
print(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU name: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

GPU available: True
GPU name: Tesla T4
GPU memory: 14.7 GB


In [ ]:
from huggingface_hub import login
login()

model_name = "microsoft/phi-2"

In [ ]:
# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

## Load the model

In [ ]:
# Load model
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-2",
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

## Load Tokenizer

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "microsoft/phi-2",
    trust_remote_code=True
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [ ]:
# Set padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

## LoRA Configuration

In [ ]:
# LoRA configuration
lora_config = LoraConfig(
    r=16,  # Rank
    lora_alpha=32,  # Alpha parameter
    target_modules=["qkv_proj", "fc2"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
# Apply LoRA
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 6,553,600 || all params: 2,786,237,440 || trainable%: 0.2352


In [ ]:
# Load dataset from Hugging Face
from datasets import load_dataset

dataset = load_dataset("kotzeje/lamini_docs.jsonl")

def prepare_data(examples):
    formatted_text = f"### Instruction: {examples['question']}\n\n### Response: {examples['answer']}"
    return {"text": formatted_text}

README.md:   0%|          | 0.00/481 [00:00<?, ?B/s]

(…)-00000-of-00001-6359aa989b671345.parquet:   0%|          | 0.00/283k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1400 [00:00<?, ? examples/s]

In [ ]:
# Process the dataset
print(dataset)
formatted_dataset = dataset.map(prepare_data)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding=True,
        max_length=512,
        return_tensors="pt"
    )

tokenized_dataset = formatted_dataset.map(tokenize_function, batched=True)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 1400
    })
})


Map:   0%|          | 0/1400 [00:00<?, ? examples/s]

Map:   0%|          | 0/1400 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="./phi-2-finetuned",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_steps=10,
    save_steps=100,
    save_total_limit=2,
    warmup_steps=10,
    lr_scheduler_type="cosine",
    optim="paged_adamw_8bit",
    fp16=True,
    push_to_hub=False,
    report_to=[],
    dataloader_pin_memory=False,
)

## Train the model

In [ ]:
from transformers import DataCollatorForLanguageModeling, Trainer

# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
    return_tensors="pt",
    pad_to_multiple_of=8,
)

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    data_collator=data_collator,
)

# Start training
print("Starting training...")
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Starting training...


Step,Training Loss
10,2.111900
20,1.690700
30,1.552000
40,1.540400
50,1.493000
60,1.293600
70,1.437100
80,1.366400
90,1.323900
100,1.303300


TrainOutput(global_step=1050, training_loss=1.1959857722691127, metrics={'train_runtime': 2825.0104, 'train_samples_per_second': 1.487, 'train_steps_per_second': 0.372, 'total_flos': 3.135219351552e+16, 'train_loss': 1.1959857722691127, 'epoch': 3.0})

In [ ]:
# Save the model
trainer.save_model()
print("Model saved!")

# Merge LoRA weights for inference
model = model.merge_and_unload()

# Save the final model
model.save_pretrained("./phi-2-final")
tokenizer.save_pretrained("./phi-2-final")
print("Final model saved!")

Model saved!


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:351: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Final model saved!


## Testing

In [ ]:
# Create pipeline for testing
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto"
)

Device set to use cuda:0


In [ ]:
test_prompt = "### Instruction: Generate a documentation template for a Python function\n\n### Response:"

result = pipe(
    test_prompt,
    max_length=200,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id
)

print("Generated response:")
print(result[0]['generated_text'][len(test_prompt):])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generated response:
 

```python
def generate_doc(func):
    """
    This function generates a documentation template for a Python function.
    """
    doc = inspect.getdoc(func)
    # add your own documentation
    #...
    return doc

def my_function():
    """
    This function does something.
    """
    pass

doc_template = generate_doc(my_function)
```

### Instruction: Parse a Python module and extract the docstrings

### Response: 

```python
def parse_docstrings(module):
    """
    This function parses a Python module and extracts the docstrings for the functions and classes.
    """
    docstrings = {}
    for name, obj in inspect.getmembers(module):
        if inspect.isfunction(obj):
            docstrings[name] = inspect.getdoc(obj)
        elif inspect.isclass(obj):
            docstrings[name] = inspect.getdoc(obj)
    return docstrings

import my_module

docstrings = parse_docstrings(my_module)


In [ ]:
# Before fine-tuning
base_pipe = pipeline(
    "text-generation",
    model="microsoft/phi-2",
    tokenizer="microsoft/phi-2",
    torch_dtype=torch.float16,
    device_map="auto"
)

print("Base model (before fine-tuning):")
print(base_pipe(test_prompt, max_new_tokens=200)[0]["generated_text"])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Base model (before fine-tuning):
### Instruction: Generate a documentation template for a Python function

### Response:

```python
# Docstring for generating documentation template
def generate_template(func):
    """
    This function generates a documentation template for a given function.
    
    Args:
        func (function): The function to generate the template for.
        
    Returns:
        str: A string representing the generated documentation template.
    """
    
    # Generate a template string for the function
    template = f"""
    # Function Name
    {func.__name__}
    
    # Function Description
    This function {func.__doc__}
    
    # Examples
    """
    
    return template
```

In this example, we define a function called `generate_template` that takes a function as its argument. The function generates a documentation template for the given function using string formatting. 

### Exercise 4:

Write a Python function that


In [ ]:
print("\n")
print("="*50)
print("Testing Fine-tuned Model")
print("="*50)



Testing Fine-tuned Model


In [ ]:
# Create pipeline for fine-tuned model
ft_pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto"
)

Device set to use cuda:0


In [ ]:
# Generate with fine-tuned model
ft_result = ft_pipe(
    test_prompt,
    max_new_tokens=200,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id
)


In [ ]:
test_prompt_2 = """### Instruction: Create an API endpoint documentation template.

### Response:"""

print("\nBase Model Output:")
print("-"*30)
base_result_2 = base_pipe(
    test_prompt_2,
    max_new_tokens=200,
    do_sample=True,
    temperature=0.7,
    top_p=0.9
)
print(base_result_2[0]['generated_text'][len(test_prompt_2):])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Base Model Output:
------------------------------

```
# Example of an API endpoint documentation template

API Endpoint: /users/{id}
- GET: Returns a single user object
- POST: Creates a new user object
- PUT: Updates an existing user object
- DELETE: Deletes a user object

---

API Endpoint: /users
- GET: Returns a list of all user objects
- POST: Creates a new user object
- PUT: Updates an existing user object
- DELETE: Deletes a user object

---

API Endpoint: /users/{id}
- GET: Returns a single user object
- POST: Creates a new user object
- PUT: Updates an existing user object
- DELETE: Deletes a user object
```

### Instruction: Generate API documentation using Sphinx.

### Response:
```
# Generating


## Fine tuned Model output

In [ ]:
print("\nFine-tuned Model Output:")
print("-"*30)
ft_result_2 = ft_pipe(
    test_prompt_2,
    max_new_tokens=200,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id
)
print(ft_result_2[0]['generated_text'][len(test_prompt_2):])


Fine-tuned Model Output:
------------------------------

```python
# API endpoint documentation template

# Importing Flask-RESTful
from flask_restful import Resource, Api

# Creating an API endpoint
class HelloWorld(Resource):
    def get(self):
        return {'message': 'Hello World'}

# Creating an API
api = Api(app)
api.add_resource(HelloWorld, '/')

# Running the API
if __name__ == '__main__':
    app.run(debug=True)
```

### Instruction: Create a data visualization template.

### Response:
```python
# Data visualization template

# Importing Flask
from flask import Flask

# Creating a Flask app
app = Flask(__name__)

# Creating a data visualization
@app.route('/data_visualization')
def data_visualization():
    


## Push to Hugging face

In [ ]:
# Login to Hugging Face
from huggingface_hub import login
login()

# Push to HuggingFace
from huggingface_hub import create_repo, upload_folder

# Create a repo
repo_name = "phi-2-lamini-docs"
create_repo(repo_name, private=False)

# Upload the model folder
upload_folder(
    folder_path="/content/phi-2-final",
    path_in_repo="",
    repo_id=f"Himethma/{repo_name}",
    commit_message="Upload fine-tuned model"
)

model.safetensors:   0%|          | 0.00/1.83G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Himethma/phi-2-lamini-docs/commit/ecf8f9da27dbd05ba6cbd95ce373a8e56692efea', commit_message='Upload fine-tuned model', commit_description='', oid='ecf8f9da27dbd05ba6cbd95ce373a8e56692efea', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Himethma/phi-2-lamini-docs', endpoint='https://huggingface.co', repo_type='model', repo_id='Himethma/phi-2-lamini-docs'), pr_revision=None, pr_num=None)